In [1]:
# %pip install radcad

In [2]:
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

plt.figure(figsize=(10, 5))
sns.set_style("whitegrid")
pd.options.plotting.backend = "plotly"

<Figure size 720x360 with 0 Axes>

In [3]:
system_params = {
    'prey_reproduction_rate': [0.6, 0.3],
    'predator_interaction_factor': [0.0002, 0.0001],
    'prey_interaction_factor': [0.002, 0.001],
    'prey_death_rate': [0.02, 0.01],
    'predator_death_rate': [1.0, 0.5],
    'dt': [0.1]
}

initial_state = {
    'prey_population': 100,
    'predator_population': 50
}

In [4]:
import numpy as np


def p_reproduce_prey(params, substep, state_history, prev_state, **kwargs):
    born_population = prev_state['prey_population'] * params['prey_reproduction_rate'] * params['dt']
    return {'add_prey': born_population}


def p_reproduce_predators(params, substep, state_history, prev_state, **kwargs):
    born_population = prev_state['predator_population'] * prev_state['prey_population'] * params['predator_interaction_factor'] * params['dt']
    return {'add_predators': born_population}


def p_eliminate_prey(params, substep, state_history, prev_state, **kwargs):
    population = prev_state['prey_population']
    natural_elimination = population * params['prey_death_rate']
    
    interaction_elimination = population * prev_state['predator_population'] * params['prey_interaction_factor']
    
    eliminated_population = natural_elimination + interaction_elimination * params['dt']
    return {'add_prey': -1.0 * eliminated_population}


def p_eliminate_predators(params, substep, state_history, prev_state, **kwargs):
    population = prev_state['predator_population']
    eliminated_population = population * params['predator_death_rate'] * params['dt']
    return {'add_predators': -1.0 * eliminated_population}


def s_prey_population(params, substep, state_history, prev_state, policy_input, **kwargs):
    updated_prey_population = np.ceil(prev_state['prey_population'] + policy_input['add_prey'])
    return ('prey_population', max(updated_prey_population, 0))


def s_predator_population(params, substep, state_history, prev_state, policy_input, **kwargs):
    updated_predator_population = np.ceil(prev_state['predator_population'] + policy_input['add_predators'])
    return ('predator_population', max(updated_predator_population, 0))

In [5]:
state_update_blocks = [
    {
        'policies': {
            'reproduce_prey': p_reproduce_prey,
            'reproduce_predators': p_reproduce_predators,
            'eliminate_prey': p_eliminate_prey,
            'eliminate_predators': p_eliminate_predators
        },
        'variables': {
            'prey_population': s_prey_population,
            'predator_population': s_predator_population            
        }
    }

]

In [6]:
TIMESTEPS = 1000
RUNS = 1

In [7]:
import time
from radcad import Model, Simulation
from radcad.engine import run

model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

start = time.time()
data_rc = run([simulation])
end = time.time()
duration = end - start
print(duration)

ModuleNotFoundError: No module named 'radcad'

In [ ]:
df_radcad = pd.DataFrame(data_rc)
df_radcad

In [ ]:
import plotly.express as px

plt.figure(figsize=(10, 5))

df = df_radcad[df_radcad['subset'] == 1]

fig = px.scatter(
    df,
    x='prey_population',
    y='predator_population',
    color='timestep'
)
fig.show()

df.plot(
    kind='line',
    x='timestep',
    y=['predator_population','prey_population']
)